# LGBM Prediction



In [1]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/RecSys2024/'
# 3.6.0以降だとLightGBMTunerが動かない
#!pip3 install optuna==3.5.0
!pip3 install polars lightgbm pyarrow

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 89.0 MB/s eta 0:00:00


In [2]:
# ==================================================== # Library # ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import polars as pl
import joblib
import pyarrow
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm

# Ensemble

In [ ]:
weight = 0.5

In [3]:
sub1 = pl.read_parquet(f"{base_path}/output/score_lgbm_valid_frac0.6_42.parquet") # validで学習
sub2 = pl.read_parquet(f"{base_path}/output/score_lgbm_train_frac0.6_42.parquet") # trainで学習
# for final sub
sub3 = pl.read_parquet(f"{base_path}/output/score_lgbm_valid_frac0.6_52.parquet")
sub4 = pl.read_parquet(f"{base_path}/output/score_lgbm_train_frac0.6_52.parquet")
sub5 = pl.read_parquet(f"{base_path}/output/score_lgbm_valid_frac0.6_62.parquet")
sub6 = pl.read_parquet(f"{base_path}/output/score_lgbm_train_frac0.6_62.parquet")

In [8]:
sub1.head()

impression_id,user_id,article_id,score
i32,i32,i32,f64
6451339,35982,9796527,-1.256092
6451339,35982,7851321,-1.83381
6451339,35982,9798805,-1.107772
6451339,35982,9795150,0.120893
6451339,35982,9531110,-2.594622


In [9]:
sub2.head()

impression_id,user_id,article_id,score
i32,i32,i32,f64
6451339,35982,9796527,-0.270669
6451339,35982,7851321,-0.387345
6451339,35982,9798805,0.351723
6451339,35982,9795150,0.300622
6451339,35982,9531110,-2.385168


In [10]:
sub3.head()

impression_id,user_id,article_id,score
i32,i32,i32,f64
6451339,35982,9796527,-1.235184
6451339,35982,7851321,-2.12763
6451339,35982,9798805,-0.994407
6451339,35982,9795150,0.33438
6451339,35982,9531110,-2.488303


In [11]:
sub4.head()

impression_id,user_id,article_id,score
i32,i32,i32,f64
6451339,35982,9796527,-0.145756
6451339,35982,7851321,-0.258127
6451339,35982,9798805,0.270866
6451339,35982,9795150,0.241276
6451339,35982,9531110,-2.383722


In [12]:
sub5.head()

impression_id,user_id,article_id,score
i32,i32,i32,f64
6451339,35982,9796527,-1.176638
6451339,35982,7851321,-1.726971
6451339,35982,9798805,-1.250337
6451339,35982,9795150,0.350031
6451339,35982,9531110,-2.364827


In [13]:
sub6.head()

impression_id,user_id,article_id,score
i32,i32,i32,f64
6451339,35982,9796527,-0.110258
6451339,35982,7851321,-0.109132
6451339,35982,9798805,-0.167324
6451339,35982,9795150,0.26487
6451339,35982,9531110,-2.549726


In [7]:
#sub_en = sub1.join(sub2, how='left', on=["impression_id", "user_id", 'article_id']).with_columns(
#    (pl.col('score') * weight+ pl.col('score_right') * (1-weight)).alias('score_ensemble')
#    )

sub_en = sub1.join(sub2, how='left', on=["impression_id", "user_id", 'article_id']).with_columns(
    (pl.col('score') + pl.col('score_right')).alias('score')
    ).drop('score_right')
sub_en = sub_en.join(sub3, how='left', on=["impression_id", "user_id", 'article_id']).with_columns(
    (pl.col('score') + pl.col('score_right')).alias('score')
    ).drop('score_right')
sub_en = sub_en.join(sub4, how='left', on=["impression_id", "user_id", 'article_id']).with_columns(
    (pl.col('score') + pl.col('score_right')).alias('score')
    ).drop('score_right')
sub_en = sub_en.join(sub5, how='left', on=["impression_id", "user_id", 'article_id']).with_columns(
    (pl.col('score') + pl.col('score_right')).alias('score')
    ).drop('score_right')
sub_en = sub_en.join(sub6, how='left', on=["impression_id", "user_id", 'article_id']).with_columns(
    (pl.col('score') + pl.col('score_right')).alias('score')
    ).drop('score_right')
sub_en

impression_id,user_id,article_id,score
i32,i32,i32,f64
6451339,35982,9796527,-4.194596
6451339,35982,7851321,-6.443015
6451339,35982,9798805,-2.897252
6451339,35982,9795150,1.612073
6451339,35982,9531110,-14.766366
…,…,…,…
0,1225161,9792362,-10.438069
0,1225161,9788041,-7.789238
0,1225161,9790135,-12.267625


In [14]:
#sub_en = sub_en.select(['impression_id', 'user_id', 'article_id', 'score_ensemble']).rename({'score_ensemble':'score'})
sub_en = sub_en.select(['impression_id', 'user_id', 'article_id', 'score'])
sub_en

impression_id,user_id,article_id,score
i32,i32,i32,f64
6451339,35982,9796527,-4.194596
6451339,35982,7851321,-6.443015
6451339,35982,9798805,-2.897252
6451339,35982,9795150,1.612073
6451339,35982,9531110,-14.766366
…,…,…,…
0,1225161,9792362,-10.438069
0,1225161,9788041,-7.789238
0,1225161,9790135,-12.267625


# Output

In [15]:
out_path = base_path + 'feature_output'
sub_sorted = sub_en.sort("impression_id", "user_id", "score", descending=[False, False, True])
sub_sorted = sub_sorted.with_columns(pl.col('score').rank(method="ordinal", descending=True).over(["impression_id", "user_id"]).alias('rank'))

out = sub_en.join(sub_sorted[['impression_id', 'user_id', 'article_id', 'rank']], on=['impression_id', 'user_id', 'article_id'], how='left')
out = out.group_by(['impression_id', 'user_id']).agg(pl.col('rank'))
id = pl.read_parquet(f"{out_path}/test_impression.parquet").select([
    pl.col("impression_id").cast(pl.Int32),
    pl.col("user_id").cast(pl.Int32)
])
out = id.join(out, on=["impression_id", "user_id"], how='left')
out

impression_id,user_id,rank
i32,i32,list[u32]
6451339,35982,"[6, 7, … 9]"
6451363,36012,"[4, 7, … 1]"
6451382,36162,"[5, 4, … 3]"
6451383,36162,"[2, 9, … 5]"
6451385,36162,"[5, 2, … 7]"
…,…,…
0,1589163,"[232, 235, … 80]"
0,1699456,"[241, 201, … 51]"
0,635479,"[238, 240, … 61]"


In [16]:
# check beyond accuracy impression
out.filter(pl.col('impression_id') == 0)

impression_id,user_id,rank
i32,i32,list[u32]
0,1049297,"[243, 234, … 68]"
0,231624,"[242, 236, … 84]"
0,716356,"[225, 238, … 72]"
0,1440307,"[241, 226, … 71]"
0,1822406,"[242, 212, … 55]"
…,…,…
0,1589163,"[232, 235, … 80]"
0,1699456,"[241, 201, … 51]"
0,635479,"[238, 240, … 61]"


In [19]:
#file_name = f'{base_path}/output/pred_ensemble_weight_{weight}.txt'
file_name = f'{base_path}/output/pred_ensemble_weight_final.txt'
with open(file_name, 'w') as f:
    for row in out.to_dicts():
        # 一度strにしないとlistの","の間に半角スペースが入ってしまう
        rank_str = ','.join(map(str, row['rank']))
        line = f"{row['impression_id']} [{rank_str}]\n"
        f.write(line)

### Check output

In [20]:
!head {file_name}

6451339 [6,7,4,3,8,2,1,5,9]
6451363 [4,7,2,6,8,5,3,1]
6451382 [5,4,2,1,3]
6451383 [2,9,10,3,1,6,11,4,7,8,5]
6451385 [5,2,3,4,6,1,7]
6451411 [9,2,5,6,1,8,3,7,4]
6451412 [6,2,1,4,8,5,7,3]
6451423 [14,30,24,8,7,33,18,19,27,28,12,17,6,4,21,26,31,5,25,22,13,11,20,10,9,23,16,29,32,15,3,1,2]
6451425 [3,2,5,6,1,4]
6451426 [1,5,3,2,4]


In [21]:
!tail {file_name}

0 [241,187,101,239,39,26,139,127,176,205,234,199,22,118,12,166,47,174,92,175,229,115,227,154,55,75,148,157,207,94,122,128,51,200,191,20,141,59,164,184,81,65,170,42,77,1,138,16,168,131,215,159,244,33,178,95,151,247,149,66,137,190,44,5,146,201,186,54,120,49,245,103,238,108,167,84,142,6,162,53,93,76,134,214,96,48,211,85,105,213,121,41,4,160,45,126,221,210,218,109,34,158,79,129,165,194,183,171,80,250,60,14,163,179,209,88,31,212,63,119,233,110,9,237,37,189,192,208,112,150,25,124,172,181,140,72,248,196,156,240,180,182,246,230,87,67,21,113,86,64,57,102,226,243,89,35,111,70,19,125,161,62,145,11,90,58,177,152,46,136,24,30,222,52,27,235,123,217,249,10,219,71,100,173,143,18,188,104,2,23,91,40,224,185,69,3,130,36,155,153,17,216,82,29,236,68,15,99,198,74,195,32,193,204,116,202,43,133,73,144,97,114,117,38,223,7,242,98,8,28,107,135,228,203,83,169,50,225,197,132,106,232,231,147,206,56,13,78,220,61]
0 [237,199,114,153,46,30,142,104,184,212,228,197,32,123,20,179,69,198,89,176,244,122,223,166,12,93,156,1

In [22]:
!wc -l {file_name}

13536710 /content/drive/MyDrive/RecSys2024//output/pred_ensemble_weight_final.txt
